# Chapter 10: Robustness

## Item 80: Take Advantage of Each Block in `try/except/else/finally`

In [10]:
def erics_cool_file_reader(filename):
    print("Step 1: lets gooo")
    handle = open(filename)
    try:
        print("Step 2: read the file")
        return handle.read()
    finally:
        print("Step 3: close the file")
        handle.close()


# This is essentially what `with open` is doing
erics_cool_file_reader("packages.csv")

Step 1: lets gooo
Step 2: read the file
Step 3: close the file


'Sydney, truck, 25\nMelbourne, boat, 6\nBrisbane, plane, 12\nPerth, road train, 90\nAdelaide, truck, 17\n'

In [11]:
# THE ORDER OF EXECUTION

throwaway_txt = "throwaway.txt"
with open(throwaway_txt, "wb") as f:
    f.write(b"\xf1\xf2\xf3\xf4")

data = erics_cool_file_reader("foop")

Step 1: lets gooo
Step 2: read the file
Step 3: close the file


UnboundLocalError: cannot access local variable 'handle' where it is not associated with a value

Note the order of operations. An exception did occur, but "finally" happened before the exception was propagated.

So-called finally! Its a conspiracy!!!

Bonus quiz: Why did the author not put `open` in the `try/finally` block?

In [19]:
# Else blocks
# These run when an exception didn't happen
import json


def get_some_json(data, key):
    try:
        print("READING")
        res = json.loads(data)
    except ValueError:
        print("WHOOPS")
        raise  # Author raises a KeyError, possibly for the caller's benefit?
    else:
        print("Looking!")
        return res[key]


assert get_some_json('{"foo": "bar"}', "foo") == "bar"

READING
Looking!


In [21]:
# We're only catching errors with loading json, propagate out the rest
get_some_json('{"foo": "bar"}', "baz")

READING
Looking!


KeyError: 'baz'

In [22]:
# Some bad json
get_some_json('{"foo" "bar"}', "foo")

READING
WHOOPS


JSONDecodeError: Expecting ':' delimiter: line 1 column 8 (char 7)

In [23]:
# PUT THEM ALL TOGETHER

UNDEFINED = object()


def divider(path):
    print("Open er up")
    handle = open(path, "r+")

    try:
        print("Read!!!")
        data = handle.read()

        print("Parse!!!")
        data = json.loads(data)

        print("Divide!!!")
        value = data["numerator"] / data["denominator"]
    except ZeroDivisionError:
        print("File not found")
        return UNDEFINED
    else:
        print("Calculate!!!")
        data["result"] = value
        result = json.dumps(data)
        handle.seek(0)
        handle.write(result)
        return value
    finally:
        print("Close!!!")
        handle.close()

In [25]:
# happy path
temp_path = "random_data.json"

with open(temp_path, "w+") as f:
    f.write('{"numerator": 1, "denominator": 10}')

assert divider(temp_path) == 0.1

with open(temp_path, "r+") as f:
    print(f.read())

Open er up
Read!!!
Parse!!!
Divide!!!
Calculate!!!
Close!!!
{"numerator": 1, "denominator": 10, "result": 0.1}


In [27]:
# zero division path
temp_path = "random_data.json"

with open(temp_path, "w+") as f:
    f.write('{"numerator": 1, "denominator": 0}')

assert divider(temp_path) is UNDEFINED

with open(temp_path, "r+") as f:
    print(f.read())

Open er up
Read!!!
Parse!!!
Divide!!!
File not found
Close!!!
{"numerator": 1, "denominator": 0}


In [28]:
# Bad data path
temp_path = "random_data.json"

with open(temp_path, "w+") as f:
    f.write('{"numerator": 1 lol what}')

assert divider(temp_path) == 0.1

with open(temp_path, "r+") as f:
    print(f.read())

Open er up
Read!!!
Parse!!!
Close!!!


JSONDecodeError: Expecting ',' delimiter: line 1 column 17 (char 16)

## Item 81: `assert` Internal Assumptions and `raise` Missed Expectations
Note: several linters will warn you about using `assert`

In [29]:
list_a = [1, 2, 3]
assert list_a, "MUST NOT BE EMPTY!!!"

list_b = []
assert list_b, "MUST NOT BE EMPTY!!!"

AssertionError: MUST NOT BE EMPTY!!!

In [30]:
# there's also raise
class EmptyError(Exception):
    pass


list_c = []
if not list_c:
    raise EmptyError("list_c is empty")

EmptyError: list_c is empty

In [32]:
try:
    raise EmptyError("raised it")
except EmptyError as e:
    print(f"caught {e}")

caught raised it


Note that `assert` raises `AssertionError`

So, when to use what?

### `raises`
This is considered part of a function's interface. The caller is expected to do something with it

### `assert`
This is not supposed to be part of the interface. It's used to verify assumptions in an implementation. It helps new readers of code understand that code better. Self-documenting. etc. Errors raised via assert can be treated as system problems, instead of "the caller made a mistake". "Our assumption was wrong"

In [37]:
class MovieRating:
    def __init__(self, max_rating):
        assert 0 <= max_rating <= 10, (
            "Rating must be between 0 and 10"
        )  # assumption: the caller validated the max rating
        self.max_rating = max_rating

    def rate(self, rating):
        if not 0 <= rating <= self.max_rating:
            raise ValueError(
                f"Rating must be between 0 and {self.max_rating}"
            )  # error: the caller gave too high a rating

In [38]:
rater = MovieRating(11)  # our code is bugged

AssertionError: Rating must be between 0 and 10

In [39]:
rater = MovieRating(10)
rater.rate(400)  # their code is bugged

ValueError: Rating must be between 0 and 10

Still fluffy?

That's okay, this is one of those "at your own discretion" areas.

My only warning around this (and the author warns this too) is to not catch `AssertionError`! Don't do it! It defeats the point. This includes not doing a high-level `except Exception`, because `AssertionError` _is_ an `Exception`.

### Things to Remember:
- The raise statement can be used to report expected error conditions back to the caller
- Exceptions raised by a function/method are part of its explicit interface
- The assert statement should be used to verify a programmers assumptions about code
- Dont catch/handle assertion errors! They indicate a bug!!!

### Side note: custom exceptions
stuff hes talking about here hints at something I _hope_ he talks about later (item 121): when building your own API, its usually better to make your own exception hierarchy, instead of using the builtins. That way you can have a top-level `except` for your module's code, without accidentally catching stuff you shouldn't (assertion errors!!!)